Парсер вакансий на Авито с телефонами и контактами.

Выгружает объявления по ссылкам.

Последние правки кода: 2022-02-06.

## Библиотеки и настройки

In [ ]:
#Загружаю нужные библиотеки

import pandas as pd
import math

import datetime

# регулярки
import re

# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm


# библиотека для распознования текста
import pytesseract
# путь к программе OCR
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# для работы с картинками
from PIL import Image

# для декодирования двоичных данных (для картинки)
import base64

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [ ]:
# тестовая страница для Селениума

test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [ ]:
# путь к файлам

file_path = 'C:/00_Data/stom/'
proxy_list_path = 'C:/work/keys/proxy_list.txt' # путь и названия со списком прокси

In [ ]:
# загружаю список прокси
# список простой текстовый файл, который содержит прокси в формате: адрес:порт логин:пароль

proxy_list = []

with open(proxy_list_path, 'r') as f:
    for line in f:
        proxy_list += [line.split()]
        
print('Рабочих прокси — ' + str(len(proxy_list)))

In [ ]:
# номер прокси

print('Номер прокси, от 0 до ' + str(len(proxy_list) - 1))
proxy_number = int(input())

In [ ]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
#opt.add_argument(f'--proxy-server={proxy_list[proxy_number][0]}') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [ ]:
print('Login: ' + proxy_list[proxy_number][1])
print('Pass: ' + proxy_list[proxy_number][2])

In [ ]:
# запускаю драйвер, проверяю настройки

#s=Service('C:/Users/bgi/Anaconda3/chromedriver/chromedriver.exe')
#driver = webdriver.Chrome(service=s, options=opt)

driver = webdriver.Chrome(options=opt)
driver.get(test_url)
driver.maximize_window() # раскрываю окно на максимум

## Собираю данные

In [ ]:
work_urls = pd.read_excel(file_path + 'avito_urls_2022_02_06.xlsx')

In [ ]:
work_urls.shape

In [ ]:
work_urls.sample()

In [ ]:
#data = [] # список для данных

for ad_url in tqdm(work_urls.ads_url[568:]): # цикл для парсинга страниц
    sleep(3)
    driver.get(ad_url)
    
    if driver.title != 'Ошибка 404. Страница не найдена — Объявления на сайте Авито':
        
        ads_date = driver.find_element(By.CLASS_NAME, 'title-info-metadata-item-redesign').text
        
        ads_info = (driver.find_element(By.TAG_NAME, 'script[type="application/ld+json"]')
                    .get_attribute('text')
                    .replace(': ', '')
                    .replace('<br />', ' ')
                    .replace('<p>', ' ')
                    .replace('</p>', ' ')
                    .replace('",', '')
                    .replace('"', '')
                    .replace('  ', '')
             )

        # заголовок объявления
        topic = re.findall(r'title.+', ads_info)[0].replace('title', '')

        # название клиники
        clinic = re.findall(r'name.+', ads_info)[0].replace('name', '')

        # адрес клиники
        try:
            address = re.findall(r'streetAddress.+', ads_info)[0].replace('streetAddress', '')
            
        except:
            address = 'none'

        # текст объявления
        ads_text = re.findall(r'description.+', ads_info)[0].replace('description', '')

        # ожидаемая ЗП
        try:
            price = driver.find_element(By.TAG_NAME, 'span[itemprop="price"]').get_attribute('content')

        except:
            price = 'нет'

        try: # ссылка на профиль продавца
            user_url = (driver.find_element(By.CLASS_NAME, 'seller-info-name.js-seller-info-name')
                        .find_element(By.TAG_NAME, 'a').get_attribute('href'))

        except:
            user_url = 'нет'
            
            
        # телефон
        
        # кликаю телефон, открываю картинку
        try:
            driver.find_element(By.CLASS_NAME, 'item-phone.js-item-phone').click()
            
            # жду пока прогрузится картинка
            sleep(5)
            
            # закрываю
            driver.find_element(By.TAG_NAME, 'span[data-marker="phone-popup/close-button"]').click()
            
            # копирую код картинки
            pic_code = (driver.find_element(By.CLASS_NAME, 'item-phone-number.js-item-phone-number')
                        .find_element(By.TAG_NAME, 'img').get_attribute('src'))
            
            # записываю код картинки в файл
            with open('phone.png', "wb") as file_pic:
                file_pic.write(base64.decodebytes(pic_code[22:].encode()))
            
            # загружаю картинку в переменную
            img_phone = Image.open('phone.png')
            
            # распознаю
            phone_number = pytesseract.image_to_string(img_phone)
                
        except:
            phone_number = 'none'
            
            
        # добавляю в список
        data.append([topic, clinic, address, phone_number, ads_date, ads_text, price, user_url, ad_url])
        
        
    else:
        print(driver.title)

In [ ]:
head_list = ['topic', 'clinic', 'address', 'phone_number', 'ads_date', 'ads_text', 'price', 'user_url', 'ad_url']
work = pd.DataFrame(data, columns=head_list)

In [ ]:
# Записываю в файл

work.to_excel(file_path + 'avito_2022_02_06.xlsx', encoding='utf-8', index=False)

In [ ]:
# закрываю Селениум
driver.close()